In [1]:
import torch
from torchvision import transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch import nn, optim


c:\Users\Yashwanth\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
import torch
from torchvision import transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torch import nn, optim

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transform to preprocess the input images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset using ImageFolder
train_dataset = ImageFolder(r'C:\Users\Yashwanth\leaf\trainig', transform=transform)
test_dataset = ImageFolder(r'C:\Users\Yashwanth\leaf\testing', transform=transform)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load pre-trained ResNet-50 model
model = resnet50(pretrained=True)

# Move the model to GPU
model = model.to(device)

# Modify the last fully connected layer for our classification task
num_classes = 3  # Number of classes (corn, rice, wheat)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the modified model to GPU
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        # Move data to GPU
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Testing loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            # Move data to GPU
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'pretrained_image_classification_model.pth')


Epoch 1/5, Loss: 0.0416, Accuracy: 0.7121
Epoch 2/5, Loss: 1.2252, Accuracy: 0.8485
Epoch 3/5, Loss: 0.5352, Accuracy: 0.6818
Epoch 4/5, Loss: 0.4656, Accuracy: 0.6515
Epoch 5/5, Loss: 0.4571, Accuracy: 0.9394


In [24]:
import torch
from torchvision import transforms
from PIL import Image
from torchvision.models import resnet50
from torch import nn

# Define the transform to preprocess the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the pre-trained ResNet-50 model
model = resnet50(pretrained=True)

# Modify the last fully connected layer for our classification task
num_classes = 3  # Number of classes (corn, rice, wheat)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Load the trained model weights
model.load_state_dict(torch.load('pretrained_image_classification_model.pth'))
model.eval()

# Load and preprocess the external image
image_path = r"C:\Users\Yashwanth\Downloads\download (1).jpeg" # Replace with the path to your external image
image = Image.open(image_path).convert('RGB')
input_image = transform(image).unsqueeze(0)

# Perform inference
with torch.no_grad():
    output = model(input_image)

# Get the predicted class
_, predicted_class = torch.max(output.data, 1)

# Define class names
class_names = ['corn', 'rice', 'wheat']

# Print the result
predicted_label = class_names[predicted_class.item()]
print(f"The predicted class is: {predicted_label}")


The predicted class is: corn
